In [1]:
import sys

sys.path.insert(0, "..")

import random
import numpy as np
from sklearn.preprocessing import LabelEncoder

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch
import pyro
import foraging_toolkit as ft
import time

import torch.nn.functional as F
import pyro.distributions as dist
import pyro.optim as optim
from pyro.nn import PyroModule
from pyro.infer.autoguide import (
    AutoNormal,
    AutoDiagonalNormal,
    AutoMultivariateNormal,
    init_to_mean,
    init_to_value,
)
from pyro.contrib.autoguide import AutoLaplaceApproximation
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import Predictive
from pyro.infer import MCMC, NUTS

import os
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
derivation_logger = logging.getLogger(__name__)

In [2]:

subset_starts = 420
subset_ends = 480

#1800 seconds overall

locust = ft.load_and_clean_locust(path = "locust_data/15EQ20191202_tracked.csv",
                                  desired_frames = 900,
                                  grid_size=45,
                                  rewards_x = [0.68074, -0.69292],
                                  rewards_y = [-0.03068, -0.03068],
                                  subset_starts= subset_starts,
                                  subset_ends = subset_ends,)

loc_subset = locust['subset']
loc_all = locust['all_frames']

original_frames: 45000
original_shape: (675000, 4)
resulting_frames: 900
resulting_shape: (13500, 4)
min_time 1
max_time 900


In [3]:
def derive_sampled_predictors(
    sim,
    rewards_decay=0.5,
    visibility_range=10,
    getting_worse=1.5,
    optimal=4,
    proximity_decay=1,
    generate_communicates=True,
    info_time_decay=3,
    info_spatial_decay=0.15,
    finders_tolerance=2,
    time_shift=0,
    sampling_rate = .1, 
    random_seed = 42
):
    grid = ft.generate_grid(sim.grid_size)

    grid = grid.sample(frac=sampling_rate, random_state=random_seed)

    tr = ft.rewards_to_trace(
        sim.rewards,
        sim.grid_size,
        sim.num_frames,
        rewards_decay,
        time_shift=time_shift,
        grid = grid,
    )

    sim.traces = tr["traces"]
    sim.tracesDF = tr["tracesDF"]
    derivation_logger.info("traces done")

    vis = ft.construct_visibility(
        sim.birds,
        sim.grid_size,
        visibility_range=visibility_range,
        time_shift=time_shift, grid = grid,
    )
    sim.visibility_range = visibility_range
    sim.visibility = vis["visibility"]
    sim.visibilityDF = vis["visibilityDF"]
    derivation_logger.info("visibility done")


    prox = ft.generate_proximity_score(
        sim.birds,
        sim.visibility,
        visibility_range=visibility_range,
        getting_worse=getting_worse,
        optimal=optimal,
        proximity_decay=proximity_decay,
        time_shift=time_shift,
    )
    sim.getting_worse = getting_worse
    sim.optimal = optimal
    sim.proximity_decay = proximity_decay

    sim.proximity = prox["proximity"]
    sim.proximityDF = prox["proximityDF"]
    derivation_logger.info("proximity done")

    ft.add_how_far_squared_scaled(sim)
    derivation_logger.info("how_far done")

    sim.derivedDF = (
        sim.tracesDF.merge(sim.visibilityDF, how="inner")
        .merge(sim.proximityDF, how="inner")
        .merge(sim.how_farDF, how="inner")
    )
    derivation_logger.info("derivedDF done")

    if generate_communicates:
        derivation_logger.info("starting to generate communicates")
        com = ft.generate_communicates(
            sim,
            info_time_decay,
            info_spatial_decay,
            finders_tolerance=finders_tolerance,
            time_shift=time_shift,
            grid=grid,
        )
        sim.communicates = com["communicates"]
        sim.communicatesDF = com["communicatesDF"]

        sim.derivedDF = sim.derivedDF.merge(sim.communicatesDF, how="left")

        sim.derivedDF["communicate"].fillna(0, inplace=True)

        derivation_logger.info("communicates done")

    pd.set_option("mode.chained_assignment", None)
    sim.rewardsDF.loc[:, "time"] = sim.rewardsDF["time"] - time_shift
    sim.birdsDF.loc[:, "time"] = sim.birdsDF["time"] - time_shift
    sim.tracesDF.loc[:, "time"] = sim.tracesDF["time"] - time_shift
    sim.visibilityDF.loc[:, "time"] = sim.visibilityDF["time"] - time_shift
    sim.proximityDF.loc[:, "time"] = sim.proximityDF["time"] - time_shift
    sim.how_farDF.loc[:, "time"] = sim.how_farDF["time"] - time_shift
    sim.communicatesDF.loc[:, "time"] = sim.communicatesDF["time"] - time_shift
    sim.derivedDF.loc[:, "time"] = sim.derivedDF["time"] - time_shift

    return sim

# start_time = time.time()
# loc_subset = derive_sampled_predictors(loc_subset)
# end_time = time.time()
# print("time taken", end_time - start_time)

start_time = time.time()
loc_all = derive_sampled_predictors(loc_all)
end_time = time.time()
print("time taken", end_time - start_time)
#loc_subset.tracesDF.head()
# print("vis shape", loc_subset.visibilityDF.shape,
#       "prox_shape", loc_subset.proximityDF.shape,
#       "how_far_shape", loc_subset.how_farDF.shape,
#       "communicates_shape", loc_subset.communicatesDF.shape,)


2023-09-01 14:16:18,583:  traces done
2023-09-01 14:16:51,374:  visibility done
2023-09-01 14:17:39,442:  proximity done
2023-09-01 14:17:53,824:  how_far done
2023-09-01 14:17:55,063:  derivedDF done
2023-09-01 14:17:55,064:  starting to generate communicates
2023-09-01 14:20:54,720:  communicates done


time taken 278.6186475753784
